## Introduction

In this chapter, we are going to explore how you can structure your data processing code, such that they are aware of what "valid" inputs look like.

## Schema Validation

The key idea we will be introducing here is "schema validation".

Before we go on, though, we should clarify some terms.

### Schemas

Firstly, what's a **schema**? From the [Wikipedia entry on database schemas](https://en.wikipedia.org/wiki/Database_schema):

> The database schema of a database is its structure described in a formal language supported by the database management system (DBMS). The term "schema" refers to the organization of data as a blueprint of how the database is constructed (divided into database tables in the case of relational databases).

For the purpose of this handbook, a schema can be described as:

> A declaration of what columns are expected to exist in a table: their names, data types, and valid ranges for each entry.

### Schema Validation

Next up, what about **schema validation**? Schema validation here refers to the act of checking that our data matches the schema that it is being checked against. This checking can be done for the input to a function and for the function's output too.

In other words, in our data processing functions, we check that the dataframe that we're handling follows the declared assumptions of what our data _ought_ to look like.

## When do we do schema validation?

Here are a few situations where schema validation is an important thing to do.

### At the data collection phase

If you're lucky, at the data collection phase, schema validation is already being performed, regardless of whether your data are being written to log files, added directly into a SQL database, or written in batches as parquet files. If your data are being written directly to a SQL database, then schema validation is already being performed, to the degree of stringency that the tables' columns are being specified!

### At the data transformation phase

If you're still lucky, a team of data engineers is helping you structure your data upstream of your use for analysis. Usually this means the data are being stored in a database, which has a SQL schema attached to it.

### At the analysis phase

Regardless of whether or not your data are being schema-fied upstream to when it gets to you, you should definitely take the time to walk carefully through the data and declare the expected schema that you need for your Python-based data processing functions.

This is the phase where you, the data scientist, can take action. In particular, I am going to show you the basics of how to declare pandas dataframe schemas using the Python package called `pandera`.

## DataFrame schemas with `pandera`

`pandera` is statistical validation package for pandas dataframes. Compared to alternatives that I have seen, `pandera` looks the most "natural" in its usage idioms. It is also very lightweight, focuses on doing one and only one thing well, and is very easily incorporated into a wide variety of data processing workflows.

Nothing beats learning by doing. As such, to help illustrate how to use `pandera`, I've designed an example that you can walk through to get a feel for how to use `pandera`. After that, there are a few more exercises that you can use to get more practice.

### Example: Declaring schemas with `pandera`

At the heart of `pandera` is the declaration of a dataframe schema.

Here is a plain text description of a dataframe we will be working with for this exercise, and its corresponding dataframe schema.

> Our dataframe has three columns:
> - `name`: a string column, with no nulls expected,
> - `age`: an integer column, positive only, nulls possibly present, but required for calculations,
> - `home_province`: a string column, and should be one of the 10 provinces or 3 territories of Canada, in its abbreviated form. Should not be null.

In [18]:
from pandera import Column, DataFrameSchema, Check
import pandera as pa

valid_provinces = ["AB", "BC", "MB", "NB", "NL", "NT", "NS", "NU", "ON", "PE", "QC", "SK", "YK"]

canadians_schema = DataFrameSchema(
    columns={
        "name": Column(pa.String, nullable=False),
        "age": Column(pa.Int, checks=[Check.greater_than(0)], nullable=True, required=True),
        "home_province": Column(pa.String, checks=[Check.isin(valid_provinces)], nullable=False)
    }
)

Now, let's say we have a function that accepts data of a bunch of Canadians,
and tries to calculate the mean age of individuals per province:

```python
def mean_age(df):
    return df.groupby("home_province").mean()
```

This function is one that we might consider "brittle", because there are no checks in the function that encode what we might expect to see for the dataframe `df`.

However, with `pandera`'s `check_input` decorator, we can validate the input data against the `canadians_schema` object.

In [19]:
from pandera import check_input
import pandas as pd

@check_input(schema=canadians_schema)
def mean_age(df):
    return df.groupby("home_province").mean()

And then when fed _invalid_ Canadian data:

In [20]:
canadians_bad_data = pd.DataFrame({
    "age": [0, 10, 20, 30],
    "name": ["adrian", "becky", "charlie", "david"],
    "home_province": ["AB", "BC", "PEI", "NU"]
})

mean_age(canadians_bad_data)

SchemaError: error in check_input decorator of function 'mean_age': <Schema Column: 'age' type=int> failed element-wise validator 0:
<Check _greater_than: greater_than(0)>
failure cases:
   index  failure_case
0      0             0

_Voila!_ `pandera` told us exactly which row in the dataframe failed the schema check.
If this is the responsibility of our data provider,
we can now go and complain to them that the dataframe has errors,
and show them the error message.

On the other hand, if you have good data:

In [21]:
canadians_good_data = pd.DataFrame({
    "age": [5, 10, 20, 30],
    "name": ["adrian", "becky", "charlie", "david"],
    "home_province": ["AB", "BC", "PE", "NU"]
})

mean_age(canadians_good_data)

,age
home_province,
AB,5
BC,10
NU,30
PE,20


`pandera` doesn't error out, and instead lets the calculation proceed as per normal.

## `pandera` usage tips

Here's some tips that I have for using `pandera`.

### Store schemas in its own `.py` file

I usually store them in `src/data/schemas.py`, so that the Python object is importable later on, and I can use it anywhere.

### Use the default checks when possible

`pandera` comes with a [rich library of built-in checks](https://pandera.readthedocs.io/en/stable/generated/pandera.Check.html#pandera.Check). I'd recommend using these wherever possible.

### Custom checks can be implemented

If you so choose to, your custom checks can be implemented easily by writing a custom function that takes in a pandas Series and returns a boolean Series or a single boolean. For example, `Check.greater_than(0)` can also be expressed as `lambda s: s > 0`.

## Exercises with `pandera`

Now, it's your turn to get practice writing `pandera` schemas. Give it a shot using the following datasets and data preprocessing functions.

### Exercise: Boston budget summarization

In this exercise, we are going to summarize data from the City of Boston's budget.

To get the columns of a DataFrame object `df`, call `df.columns`. This is a list-like object that can be iterated over.

In [ ]:
df.columns

## YAML Files

Describe data in a human-friendly & computer-readable format. The `environment.yml` file in your downloaded repository is also a YAML file, by the way!

Structure:

```yaml
key1: value
key2:
- value1
- value2
- subkey1:
    - value3
```

Example YAML-formatted schema:

```yaml
filename: boston_budget.csv
column_names:
- "Fiscal Year"
- "Service (cabinet)"
- "Department"
- "Program #"
...
- "Fund"
- "Amount"
```

YAML-formatted text can be read as dictionaries.

In [ ]:
spec = """
filename: boston_budget.csv
columns:
- "Fiscal Year"
- "Service (Cabinet)"
- "Department"
- "Program #"
- "Program"
- "Expense Type"
- "ACCT #"
- "Expense Category (Account)"
- "Fund"
- "Amount"
"""

In [ ]:
import yaml
metadata = yaml.load(spec)
metadata

You can also take dictionaries, and return YAML-formatted text.

In [ ]:
print(yaml.dump(metadata))

By having things YAML formatted, you preserve human-readability and computer-readability simultaneously. 

Providing metadata should be something already done when doing analytics; YAML-format is a strong suggestion, but YAML schema will depend on use case.

Let's now switch roles, and pretend that we're on side of the "analyst" and are no longer the "data provider". 

How would you check that the columns match the spec? Basically, check that every element in `df.columns` is present inside the `metadata['columns']` list.

## Exercise

Inside `test_datafuncs.py`, write a utility function, `check_schema(df, meta_columns)` that tests whether every column in a DataFrame is present in some metadata spec file. It should accept two arguments:

- `df`: a `pandas.DataFrame`
- `meta_columns`: A list of columns from the metadata spec.

```python

def check_schema(df, meta_columns):
    for col in df.columns:
        assert col in meta_columns, f'"{col}" not in metadata column spec'
```

In your test file, outside the function definition, write another test function, `test_budget_schemas()`, explicitly runs a test for just the budget data.

```python
def test_budget_schemas():
    columns = read_metadata('data/metadata_budget.yml')['columns']
    df = pd.read_csv('data/boston_budget.csv')

    check_schema(df, columns)
```

Now, run the test. Do you get the following error? Can you spot the error?

```bash
    def check_schema(df, meta_columns):
        for col in df.columns:
>           assert col in meta_columns, f'"{col}" not in metadata column spec'
E           AssertionError: " Amount" not in metadata column spec
E           assert ' Amount' in ['Fiscal Year', 'Service (Cabinet)', 'Department', 'Program #', 'Program', 'Expense Type', ...]

test_datafuncs_soln.py:63: AssertionError
=================================== 1 failed, 7 passed in 0.91 seconds ===================================
```

If there is even a slight mis-spelling, this kind of check will help you pinpoint where that is. Note how the "Amount" column is spelled with an extra space. 

At this point, I would contact the data provider to correct errors like this.

It is a logical practice to keep one schema spec file per table provided to you. However, it is also possible to take advantage of YAML "documents" to keep multiple schema specs inside a single YAML file. 

The choice is yours - in cases where there are a lot of data files, it may make sense (for the sake of file-system sanity) to keep all of the specs in multiple files that represent logical groupings of data.

## Exercise: Write `YAML` metadata spec.

Put yourself in the shoes of a data provider. Take the `boston_ei.csv` file in the `data/` directory, and make a schema spec file for that file.

## Exercise: Write test for metadata spec.

Next, put yourself in the shoes of a data analyst. Take the schema spec file and write a test for it.

## Exercise: Auto YAML Spec.

Inside `datafuncs.py`, write a function with the signature `autospec(handle)` that takes in a file path, and does the following:

- Create a dictionary, with two keys:
    - a "filename" key, whose value only records the filename (and not the full file path),
    - a "columns" key, whose value records the list of columns in the dataframe.
- Converts the dictionary to a YAML string
- Writes the YAML string to disk.

## Optional Exercise: Write meta-test

Now, let's go "meta". Write a "meta-test" that ensures that every CSV file in the `data/` directory has a schema file associated with it. (The function need not check each schema.) Until we finish filling out the rest of the exercises, this test can be allowed to fail, and we can mark it as a test to skip by marking it with an `@skip` decorator:

```python
@pytest.mark.skip(reason="no way of currently testing this")
def test_my_func():
    ...
```

## Notes

- The point here is to have a trusted copy of schema apart from data file. YAML not necessarily only way!
- If no schema provided, manually create one; this is exploratory data analysis anyways - no effort wasted!

# Datum Checks

Now that we're done with the schema checks, let's do some sanity checks on the data as well. This is my personal favourite too, as some of the activities here overlap with the early stages of exploratory data analysis.

We're going to switch datasets here, and move to a 'corrupted' version of the Boston Economic Indicators dataset. Its file path is: `./data/boston_ei-corrupt.csv`.

In [ ]:
import pandas as pd
import seaborn as sns
sns.set_style('white')
%matplotlib inline

df = pd.read_csv('data/boston_ei-corrupt.csv')
df.head()

### Demo: Visual Diagnostics

We can use a package called `missingno`, which gives us a quick visual view of the completeness of the data. This is a good starting point for deciding whether you need to manually comb through the data or not.

In [ ]:
# First, we check for missing data.
import missingno as msno
msno.matrix(df)

Immediately it's clear that there's a number of rows with empty values! Nothing beats a quick visual check like this one.

We can get a table version of this using another package called `pandas_summary`. 

In [ ]:
# We can do the same using pandas-summary.
from pandas_summary import DataFrameSummary

dfs = DataFrameSummary(df)
dfs.summary()

`dfs.summary()` returns a Pandas DataFrame; this means we can write tests for data completeness!

## Exercise: Test for data completeness.

Write a test named `check_data_completeness(df)` that takes in a DataFrame and confirms that there's no missing data from the `pandas-summary` output. Then, write a corresponding `test_boston_ei()` that tests the schema for the Boston Economic Indicators dataframe.

```python
# In test_datafuncs.py
from pandas_summary import DataFrameSummary
def check_data_completeness(df):
    
    df_summary = DataFrameSummary(df).summary()
    for col in df_summary.columns:
        assert df_summary.loc['missing', col] == 0, f'{col} has missing values'
        
def test_boston_ei():
    df = pd.read_csv('data/boston_ei.csv')
    check_data_completeness(df)
```

## Exercise: Test for value correctness.

In the Economic Indicators dataset, there are four "rate" columns: `['labor_force_part_rate', 'hotel_occup_rate', 'hotel_avg_daily_rate', 'unemp_rate']`, which must have values between 0 and 1.

Add a utility function to `test_datafuncs.py`, `check_data_range(data, lower=0, upper=1)`, which checks the range of the data such that:
- `data` is a list-like object.
- `data <= upper`
- `data >= lower`
- `upper` and `lower` have default values of 1 and 0 respectively.

Then, add to the `test_boston_ei()` function tests for each of these four columns, using the `check_data_range()` function.

```python
# In test_datafuncs.py
def check_data_range(data, lower=0, upper=1):
    assert min(data) >= lower, f"minimum value less than {lower}"
    assert max(data) <= upper, f"maximum value greater than {upper}"

def test_boston_ei():
    df = pd.read_csv('data/boston_ei.csv')
    check_data_completeness(df)

    zero_one_cols = ['labor_force_part_rate', 'hotel_occup_rate',
                     'hotel_avg_daily_rate', 'unemp_rate']
    for col in zero_one_cols:
        check_data_range(df['labor_force_part_rate'])
```

## Distributions

Most of what is coming is going to be a demonstration of the kinds of tools that are potentially useful for you. Feel free to relax from coding, as these aren't necessarily obviously automatable.

### Numerical Data

We can take the EDA portion further, by doing an empirical cumulative distribution plot for each data column.

In [ ]:
import numpy as np
def compute_dimensions(length):
    """
    Given an integer, compute the "square-est" pair of dimensions for plotting.
    
    Examples:
    - length: 17 => rows: 4, cols: 5
    - length: 14 => rows: 4, cols: 4
    
    This is a utility function; can be tested separately.
    """
    sqrt = np.sqrt(length)
    floor = int(np.floor(sqrt))
    ceil = int(np.ceil(sqrt))
    
    if floor ** 2 >= length:
        return (floor, floor)
    elif floor * ceil >= length:
        return (floor, ceil)
    else:
        return (ceil, ceil)
    
compute_dimensions(length=17)

assert compute_dimensions(17) == (4, 5)
assert compute_dimensions(16) == (4, 4)
assert compute_dimensions(15) == (4, 4)
assert compute_dimensions(11) == (3, 4)

In [ ]:
# Next, let's visualize the empirical CDF for each column of data.
import matplotlib.pyplot as plt

def empirical_cumdist(data, ax, title=None):
    """
    Plots the empirical cumulative distribution of values.
    """
    x, y = np.sort(data), np.arange(1, len(data)+1) / len(data)
    ax.scatter(x, y)
    ax.set_title(title)
    
data_cols = [i for i in df.columns if i not in ['Year', 'Month']]
n_rows, n_cols = compute_dimensions(len(data_cols))

fig = plt.figure(figsize=(n_cols*3, n_rows*3))
from matplotlib.gridspec import GridSpec
gs = GridSpec(n_rows, n_cols)
for i, col in enumerate(data_cols):
    ax = plt.subplot(gs[i])
    empirical_cumdist(df[col], ax, title=col)
    
plt.tight_layout()
plt.show()

It's often a good idea to **standardize** numerical data (that aren't count data). The term **standardize** often refers to the statistical procedure of subtracting the mean and dividing by the standard deviation, yielding an empirical distribution of data centered on 0 and having standard deviation of 1.

### Exercise

Write a test for a function that standardizes a column of data. Then, write the function.

**Note:** This function is also implemented in the `scikit-learn` library as part of their `preprocessing` module. However, in case an engineering decision that you make is that you don't want to import an entire library just to use one function, you can re-implement it on your own.

```python
def standard_scaler(x):
    return (x - x.mean()) / x.std()

def test_standard_scaler(x):
    std = standard_scaler(x)
    assert np.allclose(std.mean(), 0)
    assert np.allclose(std.std(), 1)
```

### Exercise

Now, plot the grid of standardized values.

In [ ]:
data_cols = [i for i in df.columns if i not in ['Year', 'Month']]
n_rows, n_cols = compute_dimensions(len(data_cols))

fig = plt.figure(figsize=(n_cols*3, n_rows*3))
from matplotlib.gridspec import GridSpec
gs = GridSpec(n_rows, n_cols)
for i, col in enumerate(data_cols):
    ax = plt.subplot(gs[i])
    empirical_cumdist(standard_scaler(df[col]), ax, title=col)
    
plt.tight_layout()
plt.show()

### Exercise

Did we just copy/paste the function?! It's time to stop doing this. Let's refactor the code into a function that can be called.

### Categorical Data

For categorical-type data, we can plot the empirical distribution as well. (This example uses the `smartphone_sanitization.csv` dataset.)

In [ ]:
from collections import Counter

def empirical_catdist(data, ax, title=None):
    d = Counter(data)
    print(d)
    x = range(len(d.keys()))
    labels = list(d.keys())
    y = list(d.values())
    ax.bar(x, y)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)

smartphone_df = pd.read_csv('data/smartphone_sanitization.csv')
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
empirical_catdist(smartphone_df['site'], ax=ax)

# Statistical Checks

- Report on deviations from normality.

## Normality?!

- The Gaussian (Normal) distribution is commonly assumed in downstream statistical procedures, e.g. outlier detection.
- We can test for normality by using a K-S test.

## K-S test

From Wikipedia:

> In statistics, the Kolmogorov–Smirnov test (K–S test or KS test) is a nonparametric test of the equality of continuous, one-dimensional probability distributions that can be used to compare a sample with a reference probability distribution (one-sample K–S test), or to compare two samples (two-sample K–S test). It is named after Andrey Kolmogorov and Nikolai Smirnov.

![](https://upload.wikimedia.org/wikipedia/commons/c/cf/KS_Example.png)

In [ ]:
from scipy.stats import ks_2samp
import numpy.random as npr

# Simulate a normal distribution with 10000 draws.
normal_rvs = npr.normal(size=10000)
result = ks_2samp(normal_rvs, df['labor_force_part_rate'].dropna())
result.pvalue < 0.05

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
empirical_cumdist(normal_rvs, ax=ax)
empirical_cumdist(df['hotel_occup_rate'], ax=ax)

## Exercise

Re-create the panel of cumulative distribution plots, this time adding on the Normal distribution, and annotating the p-value of the K-S test in the title.

In [ ]:
data_cols = [i for i in df.columns if i not in ['Year', 'Month']]
n_rows, n_cols = compute_dimensions(len(data_cols))

fig = plt.figure(figsize=(n_cols*3, n_rows*3))
from matplotlib.gridspec import GridSpec
gs = GridSpec(n_rows, n_cols)
for i, col in enumerate(data_cols):
    ax = plt.subplot(gs[i])
    test = ks_2samp(normal_rvs, standard_scaler(df[col]))
    empirical_cumdist(normal_rvs, ax)
    empirical_cumdist(standard_scaler(df[col]), ax, title=f"{col}, p={round(test.pvalue, 2)}")
    
plt.tight_layout()
plt.show()